<a href="https://colab.research.google.com/github/shirkoorshid/Machine-learning-task-2/blob/main/shir__koorshid__ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [241]:
# Load the MNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_of_classes = 10
the_shape = (28, 28, 1)

In [242]:
# Define the normalize_img function
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

# Define the add_noise function (if it's not defined elsewhere)
def add_noise(image, label):
  """Adds random noise to the image."""
  noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1)
  return image + noise, label

# Apply normalization and noise to training data
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(add_noise, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

# Apply normalization and noise to test data
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(add_noise, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [243]:
# Compile the model within the function
def create_model(learning_rate, dropout_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=the_shape),
        tf.keras.layers.Dense(40),
        tf.keras.layers.Activation('sigmoid'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(99),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(35),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(18),
        tf.keras.layers.Activation('sigmoid'),
        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
    ])
    # Compile the model within the function
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

# Train and evaluate the model
learning_rate = 0.6
dropout_rate =0.0001
model = create_model(learning_rate, dropout_rate) #model is now assigned the compiled model from the function
model.summary()

Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_43 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_204 (Dense)                    │ (None, 40)                  │          31,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_165 (Activation)          │ (None, 40)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 40)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_205 (Dense)                    │ (None, 99)                  │           4,059 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 99)                  │             396 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_166 (Activation)          │ (None, 99)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_206 (Dense)                    │ (None, 35)                  │           3,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_167 (Activation)          │ (None, 35)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_207 (Dense)                    │ (None, 18)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_168 (Activation)          │ (None, 18)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_208 (Dense)                    │ (None, 10)                  │             190 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_39 (Softmax)                 │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,193 (157.00 KB)

 Trainable params: 39,995 (156.23 KB)

 Non-trainable params: 198 (792.00 B)

In [244]:
history = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test
)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 2.3943 - sparse_categorical_accuracy: 0.1009 - val_loss: 2.3505 - val_sparse_categorical_accuracy: 0.1010
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 2.3316 - sparse_categorical_accuracy: 0.1027 - val_loss: 2.3411 - val_sparse_categorical_accuracy: 0.0892
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 2.3329 - sparse_categorical_accuracy: 0.1039 - val_loss: 2.3360 - val_sparse_categorical_accuracy: 0.1009
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.3290 - sparse_categorical_accuracy: 0.1022 - val_loss: 2.3264 - val_sparse_categorical_accuracy: 0.1135
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.3303 - sparse_categorical_accuracy: 0.1035 - val_loss: 2.3464 - val_sparse_categorical_accuracy: 0.0958
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.3327 - sparse_categorical_accuracy: 0.1010 - val_loss: 2.3197 - val_sparse_categorical_accuracy: 0.0958
Ep